In [8]:
import torch
import math

anchor:

anchors为 3x6的数组，每一行代表3个anchors的宽高

在Detect阶段，对anchors view 为 3x3x2的tensor,并除以对应层的stride

混合精度训练：

在训练阶段，使用torch中的amp模块来进行智能混合精度训练，这一部分见[Automatic mixed precision training](https://pytorch.org/docs/stable/amp.html?highlight=amp#module-torch.cuda.amp)

Yolov5中的loss部分
- 类别和目标均采用二元交叉熵损失，并由各自的权重负责
- 坐标使用CIoU损失

初始化部分：

balance为每个检测层的权重； p3 p5 p7

stride 8 16 32

build_targets:

见代码部分



In [ ]:
class ComputeLoss:
    # Compute losses
    def __init__(self, model, autobalance=False):
        super(ComputeLoss, self).__init__()
        device = next(model.parameters()).device  # get model device
        h = model.hyp  # hyperparameters

        # Define criteria
        BCEcls = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([h['cls_pw']], device=device))
        BCEobj = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([h['obj_pw']], device=device))

        # Class label smoothing https://arxiv.org/pdf/1902.04103.pdf eqn 3
        self.cp, self.cn = smooth_BCE(eps=h.get('label_smoothing', 0.0))  # positive, negative BCE targets

        # Focal loss
        g = h['fl_gamma']  # focal loss gamma
        if g > 0:
            BCEcls, BCEobj = FocalLoss(BCEcls, g), FocalLoss(BCEobj, g)

        det = model.module.model[-1] if is_parallel(model) else model.model[-1]  # Detect() module
        self.balance = {3: [4.0, 1.0, 0.4]}.get(det.nl, [4.0, 1.0, 0.25, 0.06, .02])  # P3-P7
        self.ssi = list(det.stride).index(16) if autobalance else 0  # stride 16 index
        self.BCEcls, self.BCEobj, self.gr, self.hyp, self.autobalance = BCEcls, BCEobj, model.gr, h, autobalance
        for k in 'na', 'nc', 'nl', 'anchors':
            setattr(self, k, getattr(det, k))

    def __call__(self, p, targets):  # predictions, targets, model
        device = targets.device
        lcls, lbox, lobj = torch.zeros(1, device=device), torch.zeros(1, device=device), torch.zeros(1, device=device)
        tcls, tbox, indices, anchors = self.build_targets(p, targets)  # targets

        # Losses
        for i, pi in enumerate(p):  # layer index, layer predictions
            b, a, gj, gi = indices[i]  # image, anchor, grid y, grid x
            tobj = torch.zeros_like(pi[..., 0], device=device)  # target obj

            n = b.shape[0]  # number of targets
            if n:
                # 取预测结果中在targets位置的值
                ps = pi[b, a, gj, gi]  # prediction subset corresponding to targets

                # Regression
                pxy = ps[:, :2].sigmoid() * 2. - 0.5  # sigmoid *2 -0.5 取值范围(-0.5,1.5) 表示预测的中心点偏移数值
                pwh = (ps[:, 2:4].sigmoid() * 2) ** 2 * anchors[i]  # (sigmoid *2)^2 * anchors 范围(0,4) * anchors 表示宽高
                pbox = torch.cat((pxy, pwh), 1)  # predicted box
                iou = bbox_iou(pbox.T, tbox[i], x1y1x2y2=False, CIoU=True)  # iou(prediction, target)
                lbox += (1.0 - iou).mean()  # iou loss

                # Objectness
                tobj[b, a, gj, gi] = (1.0 - self.gr) + self.gr * iou.detach().clamp(0).type(tobj.dtype)  # iou ratio

                # Classification
                if self.nc > 1:  # cls loss (only if multiple classes)
                    t = torch.full_like(ps[:, 5:], self.cn, device=device)  # targets
                    t[range(n), tcls[i]] = self.cp
                    lcls += self.BCEcls(ps[:, 5:], t)  # BCE

                # Append targets to text file
                # with open('targets.txt', 'a') as file:
                #     [file.write('%11.5g ' * 4 % tuple(x) + '\n') for x in torch.cat((txy[i], twh[i]), 1)]

            obji = self.BCEobj(pi[..., 4], tobj)
            lobj += obji * self.balance[i]  # obj loss
            if self.autobalance:
                self.balance[i] = self.balance[i] * 0.9999 + 0.0001 / obji.detach().item()

        if self.autobalance:
            self.balance = [x / self.balance[self.ssi] for x in self.balance]
        lbox *= self.hyp['box']
        lobj *= self.hyp['obj']
        lcls *= self.hyp['cls']
        bs = tobj.shape[0]  # batch size

        loss = lbox + lobj + lcls
        return loss * bs, torch.cat((lbox, lobj, lcls, loss)).detach()

    def build_targets(self, p, targets):
        """
        建立训练用的与标签匹配的anchors
        Args:
            p(list(tensor)):pred,多个层的输出，每个层的输出为(batch_size,num_anchors, h,w,nc)
            targets(tensor): shape (num_labels, 6) where 6 contains (batch_id,class,xywh)

        Returns:
        """
        # Build targets for compute_loss(), input targets(image,class,x,y,w,h)
        na, nt = self.na, targets.shape[0]  # number of anchors, targets
        tcls, tbox, indices, anch = [], [], [], []
        gain = torch.ones(7, device=targets.device)  # normalized to grid space gain
        # 将anchors中的每个anchor索引(0,1,2)复制nt份，nt为targets的数量 得到shape (na,nt)
        ai = torch.arange(na, device=targets.device).float().view(na, 1).repeat(1, nt)  # same as .repeat_interleave(nt)
        # 首先对targets重复(na,1,1)遍，得到shape (na,nt,6),然后将其与ai进行拼接，得到(na,nt, 7)，其中7为target,anchor_index
        # 这样，即对targets复制成了na份，每个anchor在初始时都对应所有的targets
        targets = torch.cat((targets.repeat(na, 1, 1), ai[:, :, None]), 2)  # append anchor indices

        # anchor的偏差， 这里为5个位置的偏差， [0,0], [0.5,0],...
        # 对之后的预测进行偏移，使得坐标预测范围为[-0.5,1.5]
        g = 0.5  # bias
        off = torch.tensor([[0, 0],
                            [1, 0], [0, 1], [-1, 0], [0, -1],  # j,k,l,m
                            # [1, 1], [1, -1], [-1, 1], [-1, -1],  # jk,jm,lk,lm
                            ], device=targets.device).float() * g  # offsets

        for i in range(self.nl): # 对每一检测层
            # anchors shape (nl, na, 2)
            anchors = self.anchors[i]
            gain[2:6] = torch.tensor(p[i].shape)[[3, 2, 3, 2]]  # xywh gain = 1 * feat_size

            # Match targets to anchors
            t = targets * gain #targets*对应层的坐标增益(feat_size)得到标签在对应层的实际坐标
            if nt:
                # Matches
                # t[] (3,36,2) anchors[:,None] (3,1,2) => 3,36,2
                r = t[:, :, 4:6] / anchors[:, None]  # wh ratio 计算target比上anchors的wh ratio,
                # 计算w/w' h/h'中的长边/短边的比值，并取max(w ratio, h ratio)作为targets/anchors的比值，如果小于阈值，则说明匹配
                j = torch.max(r, 1. / r).max(2)[0] < self.hyp['anchor_t']  # compare
                # j = wh_iou(anchors, t[:, 4:6]) > model.hyp['iou_t']  # iou(3,n)=wh_iou(anchors(3,2), gwh(n,2))
                t = t[j]  # filter

                # Offsets
                gxy = t[:, 2:4]  # grid xy label的中心坐标
                gxi = gain[[2, 3]] - gxy  # inverse # 反向后，当前点与对边的距离
                # 这里判断中心点落在cell里面的哪个方向，并取该方向相邻的两个cell作为正样本anchor
                j, k = ((gxy % 1. < g) & (gxy > 1.)).T # gt targets不在四个边上，且在中心点左边或上方
                l, m = ((gxi % 1. < g) & (gxi > 1.)).T # gt targets不在四个边上，且在中心点右边或下方
                j = torch.stack((torch.ones_like(j), j, k, l, m)) # shape (5,num_matched_gts) (1,中心点左边,上，右，下）
                t = t.repeat((5, 1, 1))[j] # 对targets重复5次并选择最近的三个
                # zeros 1,num_matched_gts,2 + off 5,1,2 = shape 5,num_xx,2  filter => filter,2
                offsets = (torch.zeros_like(gxy)[None] + off[:, None])[j] # 中心点xy加上偏移，并保留最近的三个，
            else:
                t = targets[0]
                offsets = 0

            # Define
            b, c = t[:, :2].long().T  # image, class
            gxy = t[:, 2:4]  # grid xy
            gwh = t[:, 4:6]  # grid wh
            gij = (gxy - offsets).long() # 得到gt的i，j个cell坐标
            gi, gj = gij.T  # grid xy indices

            # Append
            a = t[:, 6].long()  # anchor indices
            indices.append((b, a, gj.clamp_(0, gain[3] - 1), gi.clamp_(0, gain[2] - 1)))  # image, anchor, grid indices
            tbox.append(torch.cat((gxy - gij, gwh), 1))  # box
            anch.append(anchors[a])  # anchors
            tcls.append(c)  # class

        return tcls, tbox, indices, anch
